In [1]:
import pandas as pd

In [2]:
df=pd.read_excel(r'C:/Users/hamar/Desktop/GL4 Sem2/Data Mining/Projet Data Mining/crowdedness-at-the-campus-gym/crowdedness_hour_clean.xlsx')

In [3]:
df.head()

,number_people,day_of_week,is_weekend,is_holiday,is_start_of_semester,is_during_semester,month,hour,period_num,temperature
0,0,4,0,0,0,0,1,0,3,7
1,0,4,0,0,0,0,1,1,3,6
2,0,4,0,0,0,0,1,5,3,5
3,0,4,0,0,0,0,1,6,0,5
4,0,4,0,0,0,0,1,7,0,5


In [4]:
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
df1=df.copy()

C:\Users\hamar\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
X=df1.drop('number_people',axis=1)
y=df1.number_people

In [6]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)


In [7]:
model=linear_model.LinearRegression()
model.fit(Xtrain,ytrain)
model.score(Xtest,ytest)

0.59587516265955021

In [8]:
import numpy as np
predicted=model.predict(X)
predicted=np.round(predicted)
df['predicted']=predicted
df.head()

,number_people,day_of_week,is_weekend,is_holiday,is_start_of_semester,is_during_semester,month,hour,period_num,temperature,predicted
0,0,4,0,0,0,0,1,0,3,7,0.0
1,0,4,0,0,0,0,1,1,3,6,1.0
2,0,4,0,0,0,0,1,5,3,5,7.0
3,0,4,0,0,0,0,1,6,0,5,14.0
4,0,4,0,0,0,0,1,7,0,5,15.0


In [9]:
coeff=pd.DataFrame([X.columns,model.coef_])

In [10]:
coeff

,0,1,2,3,4,5,6,7,8
0,day_of_week,is_weekend,is_holiday,is_start_of_semester,is_during_semester,month,hour,period_num,temperature
1,-0.602015,-5.98095,-18.7808,6.9794,8.68109,-1.52385,1.63508,-1.77321,0.965336


In [11]:
coeff_temp=pd.to_numeric(coeff[8][1])
coeff_hour=pd.to_numeric(coeff[6][1])
coeff_period=pd.to_numeric(coeff[7][1])
coeff_month=pd.to_numeric(coeff[5][1])
coeff_start_sem=pd.to_numeric(coeff[3][1])
coeff_during_sem=pd.to_numeric(coeff[4][1])
coeff_day=pd.to_numeric(coeff[0][1])


In [12]:
cte = model.intercept_
cte

2.865749027145732

In [13]:
import numpy as np

In [14]:
def convertDayToNum(day):
    if (day=='Lundi') : 
        return 0
    elif (day=='Mardi') : 
        return 1
    elif(day=='Mercredi'):
        return 2
    elif(day=='Jeudi'):
        return 3
    elif (day=='Vendredi'):
        return 4
    elif (day=='Samedi'):
        return 5 
    elif(day=='Dimanche'):
        return 6

In [15]:
def convertNumToPeriod(period_num):
    if (period_num==0) : 
        return 'le matin'
    elif (period_num==1) : 
        return 'l\'après midi'
    elif(period_num==2):
        return 'le soir'
    elif(period_num==3):
        return 'tard la nuit'

In [16]:
def convertHourtoPeriod(hour):
    if (hour>=6) & (hour<12) :
        return 0
    elif (hour>=12) & (hour<18) :
        return 1
    elif (hour>=18) & (hour<22) :
        return 2
    elif (hour>=22) & (hour<6) :
        return 3

In [17]:
def OutputCrowdednessPredictions(day,temp,hour) :
    day=convertDayToNum(day)
    predict = np.round((coeff_temp*temp +coeff_hour*hour+coeff_day*day + cte))
    predict_percentage = np.round((predict/70)*100)
    nombre_min=df[df.temperature==temp][df.hour==hour][df.day_of_week==day]['predicted'].min()
    predicted_period_num = df[df.temperature==temp][df.hour==hour][df.day_of_week==day][df.predicted==nombre_min]['period_num'].min()
    period = convertHourtoPeriod(hour)
    predicted_period=convertNumToPeriod(np.round(predicted_period_num))   
    if((predict_percentage > 50) & (period != predicted_period_num) ) :
        print("La salle de sport est à {}% occupée, Elle pourrait être moins encombrée {} ".format(predict_percentage,predicted_period))
    else :
        print("La salle de sport est à {}% occupée".format(predict_percentage))

In [18]:
print ("Quel jour de la semaine ? : \n")
day=input()
print ("A quelle heure souhaitez-vous aller à la salle de sport ? : \n")
hour=int(input())
print("La température enregistrée aujoud'hui: \n ")
temperature = int(input())
OutputCrowdednessPredictions(day,temperature,hour)


Quel jour de la semaine ? : 

Lundi
A quelle heure souhaitez-vous aller à la salle de sport ? : 

22
La température enregistrée aujoud'hui: 
 
10
La salle de sport est à 69.0% occupée, Elle pourrait être moins encombrée tard la nuit 


C:\Users\hamar\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\hamar\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [19]:
from sklearn.utils import resample
np.random.seed(1)
err = np.std([model.fit(*resample(X, y)).coef_
              for i in range(1000)], 0)

In [20]:
params = pd.Series(model.coef_, index=X.columns)
print(pd.DataFrame({'effect': params.round(0),
                    'error': err}),sorted)

                      effect     error
day_of_week             -1.0  0.117043
is_weekend              -7.0  0.504865
is_holiday             -18.0  2.408219
is_start_of_semester     7.0  0.704130
is_during_semester       9.0  0.260904
month                   -2.0  0.042264
hour                     2.0  0.022840
period_num              -2.0  0.109409
temperature              1.0  0.044633 <built-in function sorted>
